In [11]:
import os
from datetime import datetime
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

In [2]:
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [3]:
time_str = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

region = os.environ['AWS_REGION'] #boto3.session.Session().region_name
boto_sess = boto3.Session()
sm = boto_sess.client('sagemaker')

s3_bucket = "s3://jbsnyder-sagemaker-us-east/"
s3_output_bucket = "s3://lilianwa-sagemaker-useast1"

base_job_name = "lilianwa-mlperf-mrcnn-lightning"
date_str = datetime.now().strftime("%d-%m-%Y")
time_str = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
job_name = f"{base_job_name}-{time_str}"

output_path = os.path.join(s3_output_bucket, "sagemaker-output", date_str, job_name)
code_location = os.path.join(s3_output_bucket, "sagemaker-code", date_str, job_name)

In [4]:
instance_type = 'ml.p4d.24xlarge'
instance_count = 1
repo = "jbsnyder"
tag = "pytorch-mrcnn"
#account = os.popen(f"aws sts get-caller-identity --region {region} --endpoint-url https://sts.{region}.amazonaws.com --query Account --output text").read().strip()
#image_uri = f"{account}.dkr.ecr.{region}.amazonaws.com/{repo}:{tag}"
image_uri = "920076894685.dkr.ecr.us-east-1.amazonaws.com/lilianwa:mlperf2lightning"

In [5]:
dist_strategy = "nccl"
# assert dist_strategy in ["smddp", "nccl"]
# For now, just using NCCL not SMDDP
assert dist_strategy in ["nccl"]
hyperparameters = {"config-file": f'configs/e2e_mask_rcnn_R_50_FPN_1x_{instance_count}_node_test.yaml'}

In [6]:
if dist_strategy=="nccl":
    distribution=None
    entry_point="launch_ddp_lightning.py"
    # hyperparameters['training_script']="aws_train_mlperf.py"
else:
    distribution={ "smdistributed": { "dataparallel": { "enabled": True } } }
    entry_point = "aws_train_mlperf.py"

In [7]:
channels = {"all_data": os.path.join(s3_bucket, "data", "yolo/"),
            "annotations": os.path.join(s3_bucket, "data", "coco", "annotations/"),
            "weights": os.path.join(s3_bucket, "data", "weights", "pt-resnet/")}

In [8]:
estimator = PyTorch(
    source_dir="./src",
    entry_point=entry_point,
    base_job_name=job_name,
    role=get_execution_role(),
    instance_count=instance_count,
    instance_type=instance_type,
    distribution=distribution,
    # volume_size=400, # Not necessary for P4d
    max_run=7200,
    hyperparameters=hyperparameters,
    image_uri=image_uri,
    output_path=os.path.join(output_path, 'training-output'),
    checkpoint_s3_uri=os.path.join(output_path, 'training-checkpoints'),
    model_dir=os.path.join(output_path, 'training-model'),
    code_location=code_location,
    input_mode='File',
)

In [9]:
estimator.fit(
    inputs=channels,
    wait=False,
    job_name=job_name,
)

In [10]:
estimator.logs()

2022-07-28 21:04:16 Starting - Starting the training job...ProfilerReport-1659042256: InProgress
...
2022-07-28 21:04:59 Starting - Preparing the instances for training.............................................
2022-07-28 21:12:41 Downloading - Downloading input data...................................................
2022-07-28 21:21:24 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ER

UnexpectedStatusException: Error for Training job lilianwa-mlperf-mrcnn-lightning-28-07-2022-21-04-14: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage ""
Command "/opt/conda/bin/python launch_ddp_lightning.py --config-file configs/e2e_mask_rcnn_R_50_FPN_1x_1_node_test.yaml", exit code: 1